In [2]:
# Importando bibliotecas numérica e matemática
import numpy as np
import math
# Importando bibliotecas para visualização
import matplotlib.pyplot as plt
# Importando bibliotecas para manipulação simbólica
from sympy import symbols, diff, lambdify, cos, sin, log
# Importando biblioteca para criar tabelas bonitas
from prettytable import PrettyTable

In [4]:
def metodo_bisseccao(f, a, b, precisao, max_iteracoes, exibir_grafico=True):
    if f(a) * f(b) >= 0:
        raise ValueError("Intervalo inválido. A função não muda de sinal no intervalo.")

    iteracao = 1
    valores_x = []
    limite_inferior = a
    limite_superior = b

    tabela = PrettyTable(['n', 'a', 'b', 'x', 'f(x)', 'f(a)', 'f(a) * f(x)'])

    while iteracao <= max_iteracoes and abs(b - a) >= precisao:
        x = (a + b) / 2
        valores_x.append(x)

        tabela.add_row([iteracao, a, b, x, f(x), f(a), f(a) * f(x)])

        if f(a) * f(x) > 0:
            a = x
        else:
            b = x

        iteracao += 1

    if exibir_grafico:
        valores_y = [f(i) for i in valores_x]
        plotar_grafico(limite_inferior, limite_superior, f, valores_x, valores_y, 'bisseccao', ultimo_ponto=(valores_x[-1], valores_y[-1]))

    print(tabela)

    print('\nRESULTADO ')
    return (a + b) / 2

def plotar_grafico(limite_inferior, limite_superior, f, pontox=None, pontosy=None, metodo=None, ultimo_ponto=None):
    x = np.linspace(limite_inferior, limite_superior, 100)
    y = f(x)

    plt.plot(x, y, label='Função', linewidth=3)

    if pontox is not None and pontosy is not None:
        plt.scatter(pontox[:-1], pontosy[:-1], color='red', label='Pontos das iterações')

    if ultimo_ponto is not None:
        plt.scatter(ultimo_ponto[0], ultimo_ponto[1], color='green', label='Solução encontrada')

    plt.axhline(0, color='black', linewidth=0.5)
    plt.axvline(0, color='black', linewidth=0.5)
    plt.grid(color='gray', linestyle='--', linewidth=0.5)

    if ultimo_ponto is None:
        plt.title(f'Gráfico da Função')
    else:
        plt.title(f'Gráfico da Função - Método {metodo}')
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.legend()

    print("\n\n")
    plt.show()
